In [15]:
from pprint import pprint 
import os
import django

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

In [2]:
from library_app.models import *  

In [3]:
from django.shortcuts import get_object_or_404
from rest_framework.response import Response
from rest_framework import status

In [4]:
from library_app.models import *  

In [30]:
def create_book_detail(data):
    
        name = data.get("name")
        authors = data.get("authors")
        book_detail = BookDetail.objects.create(name=name)
        book_detail.authors.set(authors)
        book_detail.save()
        serialized_data = BookDetailHandler._convert_to_dict(book_detail)
        return Response(
            data={
                "message": "New book detail record created",
                "data": serialized_data
            },
            status=status.HTTP_201_CREATED
        )

In [5]:
def retrieve_book_detail(book_details_uid):
    book_detail = get_object_or_404(BookDetail, uid=book_details_uid)
    serialized_data = _convert_to_dict(book_detail)  # Note: Use _convert_to_dict directly since it's a local function
    return Response(
        data={
            "message": "Book detail record retrieved",
            "data": serialized_data
        },
        status=status.HTTP_200_OK
    )

def _convert_to_dict(book_detail):
    return {
        "id": book_detail.uid,
        "name": book_detail.name,
        "authors": list(book_detail.authors.values_list('uid', flat=True))
    }

In [6]:
retrieve_book_detail(book_details_uid = "e17ea690-2bea-4445-82f4-94d3e8c0e501")

<Response status_code=200, "text/html; charset=utf-8">

In [8]:
class Book_Detail_Handler:
    
    @staticmethod
    def create_book_details(data):
        try:
            # Validate data
            if not data.get('name') or not data.get('authors'):
                return {"message": "data is missing"}

            # Create the BookDetail
            book_details = BookDetail.objects.create(name=data['name'])

            # Create or get Author instances and add to the book
            for author_name in data['authors']:
                author, created = Author.objects.get_or_create(name=author_name)
                book_details.authors.add(author)

            return {"message": "Book details saved successfully",
                     "data": data}

        except Exception as e:
            return {"message": str(e)}
      
    
    @staticmethod
    def update_book_detail(book_id, data):
        try:
            if not data.get('name') or not data.get('authors') or not book_id:
                return {"message": "data is missing"}
            
            # Find the book record based on book_id
            book_update = BookDetail.objects.get(uid=book_id)
            book_update.name = data['name']
            book_update.save()  # Save the name update first
            
            # Clear existing authors
            book_update.authors.clear()
            
            # Add new authors
            for author_name in data['authors']:
                author, created = Author.objects.get_or_create(name=author_name)
                book_update.authors.add(author)
            
            return {"message": "Book details updated successfully",
                    "data" : data
                   }


        except Exception as e:
            return {"message": str(e)}
            
     
    
    @staticmethod
    def retrieve_book_detail(book_id):
        try:
            if not book_id:
                return {"message": "book_id is missing"}
            
            #retrive the book details based on book_id
            book_retrive=BookDetail.objects.get(uid=book_id)
            
            authors=[]
            for auth in book_retrive.authors.all():
                authors.append(auth.name)
                
                
            response={
                "name":book_retrive.name,
                "authors":authors,
            }
            
            return {"message": "Book details retrive Successfully",
                     "data": response
                   }
        except Exception as e:
            return {"message": str(e)}
                
            
            
    @staticmethod        
    def delete_book_detail(book_id):
        try:
            if not book_id:
                return {"message": "book_id is missing"}
            
            #delete book details based on uid
            book_delete=BookDetail.objects.get(uid=book_id)
            
            authors = [auth.name for auth in book_delete.authors.all()]
                
            response={
                "name":book_delete.name,
                "authors":authors,
            }
            
            book_delete.delete()
            
            return {"message": "Book details deleted Successfully",
                     "data": response
                   }
        
        except Exception as e:
            return {"message": str(e)}
        
        
        

In [101]:
data = {
    "name": "Write me a love story",
    "authors": ["Ravinder Singh"]
}
response = Book_Detail_Handler.create_book_details(data)
print(response)

{'message': 'Book details saved successfully'}


In [121]:
obj=BookDetail.objects.all()
for a in obj:
    print(a.uid)

e17ea690-2bea-4445-82f4-94d3e8c0e501
5b7d1db1-a8e1-4f32-b857-55f902f72415
56f5d85c-eb1b-46ef-95c8-687bee8aec22
471caece-34cb-4553-af01-dd841bba6516
6deb3f62-ab19-4a4f-924f-31a391e1ffe5
0bf9df3c-9037-4b30-ad1e-4c003cc567e1


In [14]:
book_id='56f5d85c-eb1b-46ef-95c8-687bee8aec22'
response = Book_Detail_Handler.retrieve_book_detail(book_id)
print(response)

{'message': 'Book details retrive Successfully', 'data': {'name': 'The Da Vinci Code', 'authors': ['Dan Brown']}}


In [11]:
data = {
    "name": "Write me a love story",
    "authors": ["Ravinder Singh"]
}
book_id='56f5d85c-eb1b-46ef-95c8-687bee8aec22'
response = Book_Detail_Handler.update_book_detail(book_id, data)
print(response)

{'message': 'Book details updated successfully', 'data': {'name': 'Write me a love story', 'authors': ['Ravinder Singh']}}


In [13]:
data = {
    "name": "The Da Vinci Code",
    "authors": ["Dan Brown"]
}
book_id='56f5d85c-eb1b-46ef-95c8-687bee8aec22'
response = Book_Detail_Handler.update_book_detail(book_id, data)
print(response)

{'message': 'Book details updated successfully', 'data': {'name': 'The Da Vinci Code', 'authors': ['Dan Brown']}}


In [161]:
book_id='56f5d85c-eb1b-46ef-95c8-687bee8aec22'
response = Book_Detail_Handler.delete_book_detail(book_id)
print(response)

{'message': "['“56d85c-eb1b-46ef-95c8-687bee8aec22” is not a valid UUID.']"}
